# SME0104 - Cálculo Numérico
- Professor Antonio Castelo Filho

## Projeto 2 - Aplicações de curvas B-splines em imagens

## Membros

- Felipe Cadavez Oliveira     11208558
- Giovanni Shibaki Camargo    11796444
- João Victor Sene Araujo     11796382
- Lucas Keiti Anbo Mihara     11796472

## Aplicação 1

In [1]:
# Bibliotecas Utilizadas
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLineEdit, QHBoxLayout, QVBoxLayout, QPushButton, QSlider, QLabel
from PyQt5 import QtCore
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy import interpolate as interp

# Pontos fornecidos manualmente para a interpolação
x = [315, 463, 521, 567, 600, 634, 658, 713, 765, 819, 854, 968, 1021, 1060, 1057, 1001, 955, 973, 1012, 1065, 1116, 1110, 1076, 973, 937, 896, 820, 774, 730, 645, 589, 552, 509, 446, 380, 335, 295, 261, 220, 174, 163, 179, 258, 282, 234, 183, 172, 189, 226, 315]
y = [108, 122, 138, 155, 172, 190, 206, 216, 192, 154, 138, 146, 170, 232, 293, 365, 426, 513, 533, 571, 643, 734, 784, 805, 789, 773, 794, 852, 882, 872, 828, 784, 734, 724, 785, 809, 812, 785, 750, 699, 582, 548, 455, 415, 325, 275, 211, 162, 124, 108]
BsplineDegree = 3

class Slider(QSlider):
    minimumChanged = QtCore.Signal(int)
    maximumChanged = QtCore.Signal(int)

    def setMinimum(self, minimum):
        self.minimumChanged.emit(minimum)
        super(Slider, self).setMinimum(minimum)

    def setMaximum(self, maximum):
        self.maximumChanged.emit(maximum)
        super(Slider, self).setMaximum(maximum)

class MyApp(QWidget):
    def __init__(self):
        super().__init__()

        # Setup da interface gráfica
        self.setWindowTitle('B-splines Interpolation')
        self.window_width, self.window_height = 1920, 1080
        self.setMinimumSize(self.window_width, self.window_height)

        layout = QVBoxLayout()
        self.setLayout(layout)

        self.input1 = QLineEdit()
        self.input2 = QLineEdit()
        self.input3 = QLineEdit()
        self.button1 = QPushButton('Modify Point')
        self.button1.clicked.connect(self.update_chart)

        layout.addWidget(self.input1)
        layout.addWidget(self.input2)
        layout.addWidget(self.input3)
        layout.addWidget(self.button1)


        self.canvas = FigureCanvas(plt.Figure(figsize=(50, 30)))
        layout.addWidget(self.canvas)
        self.insert_ax()

        # Interpolação de B-spline
        tck,u = interp.splprep([x,y],k=BsplineDegree,s=0)
        u=np.linspace(0,1,num=500,endpoint=True)
        out = interp.splev(u,tck)

        x1 = np.array(out)[0]
        y1 = np.array(out)[1]

        # Geração do gráfico com os pontos XY fornecidos
        for i in range(len(x)):
            self.ax.annotate(str(i)+ "("+str(x[i])+", "+str(y[i])+")", (x[i], y[i] + 0.2), size=10, rotation=40)
        self.ax.fill_between(x1, y1)
        self.ax.plot(x, y, 'ro', out[0], out[1], 'b')
        self.ax.legend(['Area', 'Points', 'Interpolated B-spline'],loc='lower right')
        
        # Slider para selecionar o grau utilizado na Interpolação de B-spline
        self.slider = Slider(tickPosition=QSlider.TicksLeft, orientation=QtCore.Qt.Horizontal)
        slider_vbox = QVBoxLayout()
        slider_hbox = QHBoxLayout()
        slider_hbox.setContentsMargins(0, 0, 0, 0)
        slider_vbox.setContentsMargins(0, 0, 0, 0)
        slider_vbox.setSpacing(0)
        slider_vbox.addWidget(self.slider)
        self.slider.setMinimum(1)
        self.slider.setMaximum(5)
        self.slider.valueChanged.connect(self.update_degree)
        layout.addWidget(self.slider)

    # Função responsável por inserir o gráfico na interface
    def insert_ax(self):
        font = {
            'weight': 'normal',
            'size': 16
        }
        matplotlib.rc('font', **font)

        self.ax = self.canvas.figure.subplots()
        self.plot = None

    # Função responsável por atualizar o gráfico a cada alteração do grau da interpolação de B-spline
    def update_degree(self):
        degree = self.slider.value()
        try:
            degree = int(degree)
        except ValueError:
            degree = 3
        BsplineDegree = degree

        tck,u = interp.splprep([x,y],k=BsplineDegree,s=0)
        u=np.linspace(0,1,num=500,endpoint=True)
        out = interp.splev(u,tck)

        x1 = np.array(out)[0]
        y1 = np.array(out)[1]

        self.ax.clear()
        for i in range(len(x)):
            self.ax.annotate(str(i)+ "("+str(x[i])+", "+str(y[i])+")", (x[i], y[i] + 0.2), size=10, rotation=40)
        self.ax.fill_between(x1, y1)
        self.ax.plot(x, y, 'ro', out[0], out[1], 'b')
        self.ax.legend(['Area', 'Points', 'Interpolated B-spline'],loc='lower right')
        self.canvas.draw()

    # Função responsável por atualizar o gráfico a cada alteração dos pontos
    def update_chart(self):
        point = self.input1.text()
        new_x = self.input2.text()
        new_y = self.input3.text()
        try:
            point = int(point)
            new_x = int(new_x)
            new_y = int(new_y)
        except ValueError:
            point = -1
            new_x = 0
            new_y = 0

        x_position = [2]
        x[point] = new_x
        y[point] = new_y

        tck,u = interp.splprep([x,y],k=BsplineDegree,s=0)
        u=np.linspace(0,1,num=500,endpoint=True)
        out = interp.splev(u,tck)

        x1 = np.array(out)[0]
        y1 = np.array(out)[1]

        if(point > -1):
            self.ax.clear()
            for i in range(len(x)):
                self.ax.annotate(str(i)+ "("+str(x[i])+", "+str(y[i])+")", (x[i], y[i] + 0.2), size=10, rotation=40)
            self.ax.fill_between(x1, y1)
            self.ax.plot(x, y, 'ro', out[0], out[1], 'b')
            self.ax.legend(['Area', 'Points', 'Interpolated B-spline'],loc='lower right')
            self.canvas.draw()

if __name__ == '__main__':   
    app = QApplication(sys.argv)
    app.setStyleSheet('''
        QWidget {
            font-size: 20px;
        }
    ''')
    
    myApp = MyApp()
    myApp.show()

    try:
        sys.exit(app.exec_())
    except SystemExit:
        print('Closing Window...')

Closing Window...
